In [1]:
%load_ext autoreload

# --------------- #
# region: Imports #
import os
import sys
module_path = os.path.abspath('../../..')
if module_path not in sys.path:
  sys.path.insert(0, module_path)
# endregion   #
# --------------- #

In [2]:
from examples.cleanup.env import Cleanup
from examples.cleanup.agents import Agent
from examples.RPG.utils import load_config

from gem.models.iqn import iRainbowModel

from IPython.display import clear_output
from datetime import datetime

import argparse
import matplotlib.pyplot as plt
import random
import torch
import numpy as np

cfg = load_config(argparse.Namespace(config='../configs/config.yaml'))

seed = random.randint(1,100)

N_AGENTS = 1
agents: list[Agent] = []
models: list[iRainbowModel] = []
for i in range(N_AGENTS):
  models.append(
     iRainbowModel(
      state_size=(980, ),
      action_size=4,
      layer_size=250,
      epsilon=.9,
      device="mps",
      seed=seed,
      num_frames=5,
      n_step=3,
      BATCH_SIZE= 64,
      BUFFER_SIZE= 1024,
      LR=0.00025,
      TAU=.001,
      GAMMA=0.95,
      N=12,
      sync_freq=200,
      model_update_freq=4
    )
  )
  agents.append(
  Agent(cfg, appearance = cfg.agent.agent.appearance, 
    model = models[i]
          )
  )

# Set up tensorboard logging
if cfg.log:
    log_dir = os.path.abspath(f'../runs/{datetime.now().strftime("%Y%m%d-%H%m%s")}/')
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter(
        log_dir=log_dir
    )


env = Cleanup(
  cfg, agents
)

In [4]:
for agent, model in zip(agents, models):
  agent.reset()
  agent.init_replay(env)
  current_state = agent.current_state(env)
  print(current_state.shape)
  action = model.take_action(current_state)
  print(action)

(5, 980)
0
